In [1]:
import os
from dotenv import load_dotenv, find_dotenv
import google.generativeai as gemini
import pandas as pd

load_dotenv(find_dotenv())

gemini.configure(api_key=os.getenv("GEMINI_API_KEY"))

/home/deepesh/development/challange/solution/textToSpeech/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Merging all csv file
directory_path = '//home/deepesh/development/challange/solution/textToSpeech/learn/chatmodel/Test/'

all_files = os.listdir(directory_path)

csv_files = [file for file in all_files if file.endswith('.csv')]

dfs = []

for csv_file in csv_files:
    file_path = os.path.join(directory_path, csv_file)
    df = pd.read_csv(file_path)
    dfs.append(df)

In [5]:
combined_df = pd.concat(dfs, ignore_index=True)

output_path = directory_path + '/combined.csv'

combined_df.to_csv(output_path, index=False)

print(f"Combined CSV file saved at: {output_path}")

Combined CSV file saved at: //home/deepesh/development/challange/solution/textToSpeech/learn/chatmodel/Test//combined.csv


In [6]:
df = pd.read_csv(output_path)
filtered_df = df[df['Type'] == 'T']

# Get only the Utterance column
utterances = filtered_df['Utterance']

filtered_data = utterances.to_csv('data.csv', index=False)
read =  pd.read_csv("data.csv")
read

,Utterance
0,"Okay, I understand you've been having some sym..."
1,alright so you have excessive sleep you feel l...
2,And you mentioned your appetite as well.
3,how long have these symptoms been present?
4,over the last two weeks.
...,...
1117,I think youreally need to quit.
1118,"I think you're wrong. You know, it really worr..."
1119,do you want to wait for that to happen? Do you...
1120,"All right. Well, maybe it's something we can w..."


In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate

In [10]:
import csv
with open("/home/deepesh/development/challange/solution/textToSpeech/learn/chatmodel/data.csv", "r") as csvfile:
    line=""
    csv_reader = csv.reader(csvfile)
    for line in csvfile :
      line += csvfile.readline()
      print(line)

Utterance
"Okay, I understand you've been having some symptoms with a which have been bothersome. You tell me about those"

alright so you have excessive sleep you feel like what is excessive sleep
And you mentioned your appetite as well.

how long have these symptoms been present?
over the last two weeks.

"out of, say one week, how many days do you have these symptoms?"
every day?

"So I'm gonna ask you some other questions related to this. Okay, just provide me the best answer you can. In the last two weeks, have you found yourself? depressed, sad, hopeless. I know that."
Have you lost interest in activities that used to be fun for you?

Do you ever feel agitated in terms of like movement? Like you can't stop moving or find yourself trying to resist moving?
That's been happening.

Any changes in terms of your level of energy?
So the excessive sleep has affected your level of energy. How about any type of feeling of worthlessness or anything like that?

"you know, be yourself desirab

In [11]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_text(line)
chunks

['Okay. Sounds good.']

In [14]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=os.getenv("GEMINI_API_KEY"))
vector_store = FAISS.from_texts(chunks, embedding=embeddings)
vector_store.save_local("faiss_index")

In [20]:
def get_conversational_chain():

    prompt_template = """
    Converse with the patient using the context and try to give  support\n\n
    Context:\n {context}?\n
    Question: \n{question}\n

    Answer:
    """

    model = ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.7, google_api_key=os.getenv("GEMINI_API_KEY"))

    prompt = PromptTemplate(template = prompt_template, input_variables = ["context", "question"])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)

    return chain

In [21]:
embeddings = GoogleGenerativeAIEmbeddings(model = "models/embedding-001",google_api_key=os.getenv("GEMINI_API_KEY"))
new_db = FAISS.load_local("faiss_index", embeddings)
docs = new_db.similarity_search("Um, I've been having a lot of worry. Worry. Yes. A lot of worry on and off for about the past four or five months, maybe four or five months, about four or five months, maybe about six months? Yeah, round four or five, six months. And I just started back to school after being a stay at home mom for 20 years. The Kids Are All awful to college now in a way and I decided why not go back to school do something else now.")
chain = get_conversational_chain()
response = chain(
        {"input_documents":docs, "question": "Um, I've been having a lot of worry.Yes. A lot of worry on and off for about the past four or five months, maybe four or five months, about four or five months, maybe about six months? Yeah, round four or five, six months. And I just started back to school after being a stay at home mom for 20 years. The Kids Are All awful to college now in a way and I decided why not go back to school do something else now."}
        , return_only_outputs=True)

print(response)

/home/deepesh/development/challange/solution/textToSpeech/venv/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{'output_text': "I understand that you're feeling a lot of worry and anxiety. It's normal to feel this way when you're going through a big change in your life, like going back to school after being a stay-at-home mom for 20 years. It's also understandable that you're feeling overwhelmed by the responsibilities of being a student and a parent.\n\nIt's important to remember that you're not alone. Many people feel the same way when they go back to school. There are resources available to help you cope with your anxiety and stress.\n\nOne thing that can help is to talk to a counselor or therapist. They can help you identify the sources of your anxiety and develop coping mechanisms.\n\nAnother thing that can help is to take care of yourself. Make sure you're getting enough sleep, eating healthy foods, and exercising regularly. These things can help improve your mood and energy levels.\n\nFinally, it's important to be patient with yourself. It takes time to adjust to a new routine and to lea